# Getting started: `gfn` from scratch

The goal of this notebook is two-fold:

* to take you through both the intuitions of why and how to use GFlowNets
* to teach you the basic structures and components of `gfn` to train your own GFlowNets

This is neither a theoretical course on GFlowNets (see resources at the end) nor a guide on how to invent your own `gfn` components (we have [other tutorials for that](link)).

Let's get started!